In [2]:
import numpy as np
from scipy import sparse 
import matplotlib.pyplot as plt

### softmax function

In [70]:
def softmax (Z):
    e_Z = np.exp(Z)
    A = e_Z/ e_Z.sum(axis = 0)
    return A

### softmax function ổn định hơn

In [71]:
def softmax_stable(Z):
    e_Z = np.exp(Z - np.max(Z, axis = 0 , keepdims = True))
    A = e_Z / e_Z.sum(axis = 0)
    return A

In [72]:
N = 2 
d = 2 
C = 3
X = np.random.randn(d, N)
y = np.random.randint(0, 3, (N,))

In [73]:
def convert_labels(y, C = C):
    Y = sparse.coo_matrix((np.ones_like(y), 
        (y, np.arange(len(y)))), shape = (C, len(y))).toarray()
    return Y 

Y = convert_labels(y, C)

In [74]:
def cost (X,Y,W):
    A = softmax(W.T.dot(X))
    return -np.sum(Y*np.log(A))
W_init = np.random.randn(d,C)

def grad (X,Y,W):
    A = softmax((W.T.dot(X)))
    E = A - Y
    return X.dot(E.T)

def numerical_grad(X, Y, W, cost):
    eps = 1e-6
    g = np.zeros_like(W)
    for i in range (W.shape[0]):
        for j in range(W.shape[1]):
            W_p = W.copy()
            W_n = W.copy()
            W_p[i,j] += eps
            W_n[i,j] -= eps
            g[i,j] = cost(X,Y,W_p) - cost(X,Y,W_n)/(2*eps)
    return g
g1 = grad(X, Y, W_init)
g2 = numerical_grad(X, Y, W_init, cost)

print(np.linalg.norm(g1-g2))

    

9074845.956226652


In [75]:
def softmax_regression (X,Y,W_init,eta,tol = 1e-4,max_count = 10000):
    W = [W_init]
    C = W_init.shape[1]
    Y = convert_labels(y,C)
    it = 0 
    N = X.shape[1]
    d = X.shape[0]
    count = 0 
    check_w_after = 20 
    while count < max_count:
        mix_id = np.random.permutation(N)
        for i in mix_id:
            xi = X[:,i].reshape(d,1)
            yi = Y[:,i].reshape(C,1)
            ai = softmax(np.dot(W[-1].T,xi))
            W_new = W[-1] + eta*xi.dot((yi - ai).T)
            count += 1
            
            if count%check_w_after == 0:
                if np.linalg.norm(W_new - W[-check_w_after]) < tol:
                    return W
            W.append(W_new)
    return W
eta = 0.05
d = X.shape[0]
W_init = np.random.randn(d,C)
W = softmax_regression(X,Y,W_init,eta)
            
            
                         
            

In [76]:
def pred(W,X):
    A = softmax_stable(W.T.dot(X))
    return np.argmax(A,axis = 0)

In [77]:
means = [[2, 2], [8, 3], [3, 6]]
cov = [[1, 0], [0, 1]]
N = 500
X0 = np.random.multivariate_normal(means[0], cov, N)
X1 = np.random.multivariate_normal(means[1], cov, N)
X2 = np.random.multivariate_normal(means[2], cov, N)

# each column is a datapoint
X = np.concatenate((X0, X1, X2), axis = 0).T 
# extended data
X = np.concatenate((np.ones((1, 3*N)), X), axis = 0)
C = 3

original_label = np.asarray([0]*N + [1]*N + [2]*N).T

In [5]:
W_init = np.random.randn(X.shape[0], C)
W = softmax_regression(X, original_label, W_init, eta)
print(W[-1])


NameError: name 'X' is not defined